# Supplementary Figure S6: Windrose graphs

Based on: https://hatarilabs.com/ih-en/how-to-make-a-wind-rose-with-python-tutorial

In [ ]:
import pandas as pd
import numpy as np

# Note: You need to do: conda install windrose
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from math import pi
from windrose import WindroseAxes

In [ ]:
project_path = '../'

graphs_path = project_path + '../graphs/'
data_path = project_path + '../data/'

ec_fn = data_path + 'dataset.csv'

## Load data

In [ ]:
def load_data(fn, index_col=False, silent=False):
    if (not silent):
        print('-', fn.split('/')[-1])
    temp = pd.read_csv(fn, index_col=index_col)
    temp['DateTime'] = pd.to_datetime(temp['DateTime'], format='%Y-%m-%d %H:%M:%S', utc=True)
    return(temp)

In [ ]:
print("Loading data")
print("------------")

ec = load_data(ec_fn)

# Extract only PV field data
ec = ec.loc[ec['Ecosystem'].str.contains('PV')].copy()
# Re-label
ec.loc[ec['Ecosystem'] == 'PV field', 'Ecosystem'] = 'PV'
ec.loc[ec['Ecosystem'] == 'PV desert background', 'Ecosystem'] = 'Desert'

# Define seasons
ec['season'] = np.nan
ec.loc[ec['Month'] == 'March', 'season'] = 'Spring'
ec.loc[ec['Month'] == 'October', 'season'] = 'Autumn'
ec.loc[ec['Month'] == 'July', 'season'] = 'Summer'
ec.loc[ec['Month'] == 'August', 'season'] = 'Summer'

ec['location'] = ec['season'] + ' - ' + ec['Ecosystem']

print('Done...')
#print(ec.columns.values)

In [ ]:
ec['location'].unique()

In [ ]:
plt.rcParams.update({'font.size': 12})
plt.rcParams['font.family'] = 'serif'

nrows, ncols = 3, 2
fig = plt.figure(figsize=(8, 12), dpi=300)
bins = np.arange(0.01, 1.2, 0.2)
print(max(ec['uStar']))

# Categories, in order, from ec['location'].unique():
categories = ['Spring - Desert',  'Spring - PV', 'Summer - Desert',  'Summer - PV', 'Autumn - Desert', 'Autumn - PV']

i = 1
print('Preparing plot for:')
for location in categories:
    print('  ('+ chr(ord('`')+i) + ') ' + location)
    temp = ec.loc[(ec['location'] == location)]
    ax = fig.add_subplot(nrows, ncols, i, projection="windrose")
    ax.bar(temp['Wind_direction'], temp['uStar'], bins=bins, normed=True, opening=0.8, edgecolor='white')
    ax.set_title('('+ chr(ord('`')+i) + ') ' + location, pad=20)
    i+=1

# Add legend
plt.legend(title='$u_*$ [$m~s^{-1}$]', prop={'size': 10}, fontsize = 'large', loc='lower center', bbox_to_anchor=(-0.1, -0.5), ncol=3)
# Adjust subplots
plt.subplots_adjust(left=0.1,bottom=0.1,right=0.9,top=0.9,wspace=0.4,hspace=0.4)

# Save
print('Saving...')
plt.savefig(graphs_path + 'windroses.pdf')
plt.savefig(graphs_path + 'windroses.png', dpi=300)
print('Done...')

plt.show()

In [ ]:
# Show a single season

temp = ec.loc[(ec['location'] == 'Summer - PV')]

plt.rcParams.update({'font.size': 18})
plt.setp(ax.get_legend().get_texts(), fontsize='12')
plt.rcParams['font.family'] = 'serif'

ax = WindroseAxes.from_ax()
ax.bar(temp['Wind_direction'], temp['uStar'], normed=True, opening=0.8, edgecolor='white')
ax.set_legend(loc='lower right', title='$u_*$ [$m~s^{-1}$]', prop={'size': 10}, fontsize = 'large')
ax.set_title('Summer - PV')
# Change font size of legend
plt.setp(ax.get_legend().get_texts(), fontsize='12')

# Save
#plt.savefig(graphs_path + 'windroses.pdf')
#plt.savefig(graphs_path + 'windroses.png', dpi=300)